In [1]:
# Import Necessary Libraries
from dotenv import load_dotenv
import os
import base64
from requests import post, get
import json
from prettyprinter import pprint as pp

import pandas as pd

In [2]:
# Access Client ID and Secret to Access API in Hidden .env File
load_dotenv()
client_id = os.getenv("CLIENT_ID")
secret_id = os.getenv("CLIENT_SECRET")

In [3]:
def extract_token(url = "https://accounts.spotify.com/"):
    '''
    Insert Docstring
    '''
    client_id = os.getenv("CLIENT_ID")
    secret_id = os.getenv("CLIENT_SECRET")
    auth_string = f"{client_id}:{secret_id}"
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
    
    url = f"{url}api/token"
    headers = {"Authorization": "Basic " + auth_base64, "Content-Type": "application/x-www-form-urlencoded"}
    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    print(json_result)
    token = json_result["access_token"]
        
    return token

In [4]:
# Extracting Artist ID Using Token
def artist_id_search(artist_name, type, limit = 20, url = "https://api.spotify.com/"): 
    '''
    Insert Docstring
    '''
    url = f"{url}v1/search"
    query = f"?q={artist_name}&type={type}&limit={limit}"
    query_url = f"{url}{query}"
    token = extract_token()
    headers = {"Authorization": f"Bearer {token}"}
    result = get(query_url, headers= headers)
    json_result = json.loads(result.content)['artists']['items']

    if len(json_result) == 0:
       print(f"No artist with this name {artist_name} exists. Please check name and try again")
       return None
    return json_result

In [5]:
api = artist_id_search("fela","artist") 

{'access_token': 'BQDMaG8PFRhRx4DTKTnwRRUWg3vYMmyhTmnqCu80LimgszHSk9HMd6ZjvboallolSQT5eTJFNKoCX6x9jUCNnN2YVhKX36VgBdqXyff2TPUBHI2udkI', 'token_type': 'Bearer', 'expires_in': 3600}


In [6]:
href = []
ids = []
spotify = []
genre = []
total = []
name = []
popularity = []
types = []

columns = {}

def flatten(data, append=None):
    if type(data) == dict:
        
        for key in data.keys():
            if type(data[key]) in [str, int, None]:
                if key == 'href':
                    href.append(data[key])
                    columns[key] = href
                elif key == 'id':
                    ids.append(data[key])
                    columns[key] = ids
                elif key == 'name':
                    name.append(data[key])
                    columns[key] = name
                elif key == 'type':
                    types.append(data[key])
                    columns[key] = types                    
                elif key == 'popularity':
                    popularity.append(data[key])
                    columns[key] = popularity 
                elif key == 'spotify':
                    spotify.append(data[key])
                    columns[key] = spotify   
                elif key == 'total':
                    total.append(data[key])
                    columns[key] = total                    
            
            elif type(data[key]) == list:
                if sum([True for k in data[key] if type(k) == str]) == len(data[key]):
                    if key == 'genres':
                        genre.append(data[key])
                        columns[key] = genre
            else: 
                flatten(data[key])
   
    elif type(data) == list:
        for item in data:
            flatten(item)

    else:
        pass #print(f"{data} cannot be transformed at this time")

    return columns